In [1]:
import os,re
import shutil
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
from matplotlib.pylab import mpl
import struct
import serial
from pathlib import Path

mpl.rcParams['font.sans-serif'] = ['SimHei']
mpl.rcParams['axes.unicode_minus'] = False
g = 9.801538877

def move_dir(src,dest):
    if not os.path.exists(dest):
        os.makedirs(dest)
    shutil.move(src,dest)
def split_pro(data):
    split_symbol = [' ',',','，']
    for symbol in split_symbol:
        data = '#*#'.join(data.split(symbol))
    return [item for item in data.split('#*#') if item]

In [2]:

def del_empty_folder(path):
    list_delFolder = []
    for folder in os.listdir(path):
        folder_path = os.path.join(path,folder)
        if os.path.isdir(folder_path):
            del_empty_folder(folder_path)
            if not os.listdir(folder_path):
                os.rmdir(folder_path)
                list_delFolder.append(folder)
    return list_delFolder

def extract_number(filename):
    match = re.search(r'BD(\d+)#',filename)
    return int(match.group(1)) if match else float('inf')
def sort_bd_files(folder_path):
    files = os.listdir(folder_path)
    bd_files = [f for f in files if re.search(r'BD(\d+)#',f)]
    sorted_files = sorted(bd_files,key=extract_number)
    return sorted_files


In [5]:
class struct_paraCal:
    def __init__(self):
        self.init_list()
        self.init_conf()
        
    def init_list(self):
        # 初始化测试结果变量
        self.list_SL = []
        self.list_SL_FFZ = []
        self.list_BD_PPM = []
        self.list_LW = []
        self.list_LW_FFZ = []
        self.list_LW_PPM = []
        self.list_LW_STD = []
        self.list_JT = []
        self.list_JT_FFZ = []
        self.list_JT_Allan = []
        self.list_JT_10s = []
        self.list_JT_ns = []
        self.list_FXXD = []
        self.list_FXXD_FFZ = []
        self.list_FXXD_PPM = []
        self.list_FXXD_PPM = []
        self.list_FXXD_STD = []
        self.list_Kxn = []
        self.list_Kyn = []
        self.list_Kzn = []
        self.list_sqrt = []
        
    def init_conf(self):
        # 初始化全局变量
        # 当地属性
        self.longitude = 116.50271
        self.latitude = 39.73155
        self.G = 9.801538877
        # 标度因数相关
        self.SL_name = ['SL','s_average.txt']
        self.SL_sort = [0,1,2]
        self.SL_skipNum = 0
        self.SL_rShiftNum = 1
        self.SL_point_count = [10,10,10]
        self.SL_point_list = [10,50,100,150,180]
        self.SL_point_zf = [1,1,1]
        # 零位相关
        self.LW_name = ['LW','s_average.txt']
        self.LW_sort = [2,1,0]
        self.LW_skipNum = 4
        self.LW_rShiftNum = 4
        self.lw_skip = [0,0,0]
        self.ga_shift = [3,3,3]
        # 静态相关
        self.JT_name = ['JT','s_average.txt']
        self.JT_rolls_num = 10
        # 归一化处理 到°/h和m/s²
        # 加速度单位：m/s^2；角速度单位：rad/s。 温度:℃
        self.num_GxyzShift = 1
        self.Gxyz_paraMult = [3600,3600,3600]
        self.num_AxyzShift = 4
        self.Axyz_paraMult = [1,1,1]
        self.num_GxyzTempShift = 7
        self.Gtemp_paraMult = [1,1,1]
        self.num_AxyzTempShift = 10
        self.Atemp_paraMult = [1,1,1]
        self.num_saveDecimal = 6
        self.num_Hz = 200
        # 配置内容
        self.list_showMsg = []
        self.list_debugMsg = []
        self.flag_delEmptyFolder = 1
        self.flag_saveSecondAverage = 1
        self.flag_saveHzAverage = 1
        self.flag_onlySaveXYZ = 1
        self.list_GAGtAt = [
            'Turntable',
            'Gx','Gy','Gz',
            'Ax','Ay','Az',
            'GxTemp','GyTemp','GzTemp',
            'AxTemp','AyTemp','AzTemp',
            'Length','Other'
            ]
        # 尽可能多的取整数圈
        self.num_remainderSL = 36
        self.num_remainderLW = 10
        self.num_remainderMax = 3600
        
    def load_BD_file(self,paths):
        self.processing_BD_file(paths,mode='ave')
        try: self.processing_BD_file(paths,mode='ave')
        except Exception as e: self.list_showMsg.append('Exceprion [processing_BD_file]:{}'.format(e))
        try: self.processing_BD_file(paths,mode='std')
        except Exception as e: self.list_showMsg.append('Exception [processing_BD_file]:{}'.format(e))
        return
        self.load_SL_file(paths)
        return
        self.load_LW_file(paths)
        self.load_JT_file(paths)
        
    # 查看全局变量
    def show_dict(self):
        for i in self.__dict__:
            if 'list' not in i:
                print('{}\t{}'.format(i,self.__dict__[i]))
        pass
    # 初始化变量列表 将数字转列表
    def get_all2List(self,lists=None):
        if isinstance(lists,list):
            init_num_list = lists
        else:
            return False
        for item_count in range(len(init_num_list)):
            item = init_num_list[item_count]
            if isinstance(item,int):
                init_num_list[item_count] = [item,item+1,item+2]
            elif isinstance(item,list):
                for i in range(3):
                    try:item[i]
                    except:init_num_list[item_count].append(item[i-1])
            else:
                print('格式错误:{}'.format(item))
        return init_num_list
        
    # 平均处理标定数据
    def processing_BD_file(self,paths,mode='ave'):
        load_paths = Path(paths)
        if mode=='ave':
            processMode = 'ave'
            std_num = ''
        elif mode=='std':
            processMode = 'std'
            std_num = 'roll{}s_'.format(self.JT_rolls_num)
        # 删除空文件夹
        if self.flag_delEmptyFolder:
            list_delFolder = del_empty_folder(load_paths)
            for msg in list_delFolder:
                self.list_showMsg.append('删除空文件夹: {}'.format(msg))
        # 删除均值缓存文件夹
        save_path = load_paths/('all_%s'%processMode)
        if os.path.exists(save_path):
            shutil.rmtree(save_path)
        if not os.path.exists(save_path):
            os.makedirs(save_path)
            
        for plan_name in os.listdir(load_paths):
            if not os.path.isdir(load_paths/plan_name):
                continue
            if plan_name=='all_%s'%processMode:
                continue
            list_all_file = [f for f in os.listdir(load_paths/plan_name) if re.search(r'BD(\d+)#',f)]
            list_hzFile = [i for i in list_all_file if ('hz.txt' in i)]
            list_sFile = [i for i in list_all_file if ('s.txt' in i)]
            if list_hzFile :
                if self.flag_saveHzAverage:
                    save_file_hz = save_path/'{}_{}_{}hz.txt'.format(plan_name,processMode,std_num)
                    list_hzFile = sorted(list_hzFile,key=extract_number)
            if list_sFile :
                if self.flag_saveSecondAverage:
                    save_file_s = save_path/'{}_{}_{}s.txt'.format(plan_name,processMode,std_num)
                    list_sFile = sorted(list_sFile,key=extract_number)
                    
            for filename in list_hzFile+list_sFile:
                if 'hz.txt' in filename:
                    Hz = self.num_Hz
                    save_name = save_file_hz
                else:
                    Hz = 1
                    save_name = save_file_s
                try:
                    files = pd.read_csv(load_paths/plan_name/filename,sep='\\s+',header=None,skiprows=1, encoding='gb2312')
                except:
                    self.list_showMsg.append('打开文件失败: {}',format(filename))
                    continue
                if ('spd[' in filename):
                    turn_table = filename.split('spd[')[1].split(']')[0]
                    mean_length = -self.num_remainderSL*Hz*(files.shape[0]//(self.num_remainderSL*Hz))
                elif ('loc[' in filename):
                    turn_table = filename.split('loc[')[1].split(']')[0]
                    mean_length = -self.num_remainderLW*Hz*(files.shape[0]//(self.num_remainderLW*Hz))
                else:
                    turn_table = 'None'
                    mean_length = None
                if abs(mean_length)<10:
                    mean_length = None
                if mean_length:
                    if self.num_remainderMax*Hz<-mean_length:
                        mean_length = -self.num_remainderMax*Hz
                        
                        
                # 根据全局变量归一化处理数据
                init_num_list = [
                    self.num_GxyzShift,
                    self.num_AxyzShift,
                    self.num_GxyzTempShift,
                    self.num_AxyzTempShift
                ]
                init_num_list = self.get_all2List(init_num_list)
                init_para_list = [
                    self.num_GxyzShift,
                    self.num_AxyzShift,
                    self.num_GxyzTempShift,
                    self.num_AxyzTempShift
                ]
                init_para_list = self.get_all2List(init_para_list)
                # i=0
                # j=0
                # print(init_num_list)
                # print(init_num_list[j][i])
                # print(files.iloc[:,init_num_list[j][i]])
                # print(init_para_list[j][i])
                # print(files.iloc[:,init_num_list[j][i]]*init_para_list[j][i])
                # return 
                # 仅保存陀螺加表及对应温度
                if self.flag_onlySaveXYZ:
                    df = pd.DataFrame([])
                    for j in range(len(init_num_list)):
                        for i in range(3): 
                            df = pd.concat([ df,files.iloc[:,init_num_list[j][i]]*init_para_list[j][i] ],axis=1)
                # 全部保存
                else:
                    for j in range(len(init_num_list)):
                        for i in range(3):
                            files.iloc[:,init_num_list[j][i]] *= init_para_list[j][i]
                            # files.iloc[:,self.num_GxyzShift[i]] *= self.Gxyz_paraMult[i]
                            # files.iloc[:,self.num_AxyzShift[i]] *= self.Axyz_paraMult[i]
                            # files.iloc[:,self.num_GxyzTempShift[i]] *= self.Gtemp_paraMult[i]
                            # files.iloc[:,self.num_AxyzTempShift[i]] *= self.Atemp_paraMult[i]
                    df = files
                    
                # 构造新的列表
                    
                df = df.iloc[mean_length:,:]
                if processMode=='ave':
                    processData = files.mean()
                if processMode=='std':
                    processData=files.rolling(self.JT_rolls_num).mean().std()
                
                len_data = len(files)
                save_data_list = []
                save_data_list.append(str(turn_table))
                for i in range(len(processData)):
                    save_data_list.append('{:.{}f}'.format(processData[i],self.num_saveDecimal))
                save_data_list.append(str(len_data))

                # save_data = '{}\t{}\t{}\n'.format(
                #     turn_table,
                #     '\t'.join(['{:.{}f}'.format(i,self.num_saveDecimal) for i in processData]),
                #     len_data
                # )
                if not os.path.exists(save_name):
                    with open(save_name,encoding='gb2312',mode='w') as f:
                        save_title_list = []
                        for i in range(len(save_data_list)):
                            try:
                                save_title_list.append(self.list_GAGtAt[i])
                            except:
                                save_title_list.append(self.list_GAGtAt[-1])
                        f.write('\t'.join(save_title_list)+'\n')
                with open(save_name,encoding='gb2312',mode='a+') as f:
                    f.write('\t'.join(save_data_list)+'\n')
    
    # 标准差处理标定数据
    def stds_BD_file(self,paths):
        load_paths = Path(paths)
        # 删除空文件夹
        if self.flag_delEmptyFolder:
            list_delFolder = del_empty_folder(load_paths)
            for msg in list_delFolder:
                self.list_showMsg.append('删除空文件夹: {}'.format(msg))
        # 删除均值缓存文件夹
        save_path = load_paths/'all_std'
        if os.path.exists(save_path):
            shutil.rmtree(save_path)
        if not os.path.exists(save_path):
            os.makedirs(save_path)
        
        
        
        
        
    # 处理速率文件
    def load_SL_file(self,paths):
        listdir = os.listdir(paths)
        listdir = [files for files in listdir if (all(s in files for s in self.SL_name))]
        sorted_list = []
        left_list = []
        for name in listdir:
            try: sorted_list.append(name) if int(name.split(self.split_char)[self.split_loc])&False else sorted_list.append(name)
            except:left_list.append(name)
        sorted_list = sorted(
            sorted_list,
            key=lambda item:int(item.split(self.split_char)[self.split_loc])
            )
        listdir = sorted_list+sorted(left_list)
        df_list = []
        for name in listdir:
            df = pd.read_csv('{}/{}'.format(paths,name), )
        print(listdir)
        pass
    # 处理零位文件
    def load_LW_file(self,paths):
        
        pass
    # 处理静态文件
    def load_JT_file(self,paths):
        
        pass
        
test = struct_paraCal()
# test_paths = 'D:\\Data\\D501\\20241122#交付验证#D501#05\\861#05_BD - 副本\\'
test_paths = './data/'
test.load_BD_file(test_paths)
print('showMsgList:')
for msg in test.list_showMsg:
    print(msg)

showMsgList:


In [94]:
filename = 'D:\\Data\\D501\\20241122#交付验证#D501#05\\861#05_BD - 副本\\LW_26_1\\861#05_BD_BD1#loc[0.0_0.0_0.0_0.0]#94944_s.txt'
files = pd.read_csv(filename,sep='\\s+',header=None,skiprows=1, encoding='gb2312')
# files.shape[0]//10*10

In [125]:
df = pd.DataFrame()
for i in range(3):
    df = pd.concat([df,files.iloc[:,i+1]],axis=1)
df

,1,2,3
0,-0.003248,0.002640,-0.000462
1,-0.003170,0.002603,-0.000427
2,-0.003186,0.002745,-0.000455
3,-0.003168,0.002621,-0.000389
4,-0.003213,0.002668,-0.000491
5,-0.003183,0.002631,-0.000449
6,-0.003169,0.002687,-0.000454
7,-0.003175,0.002647,-0.000474
8,-0.003154,0.002678,-0.000401
9,-0.003205,0.002668,-0.000440


In [87]:
files.rolling(1).mean().std()

0     0.000000
1     0.000022
2     0.000034
3     0.000024
4     0.000121
5     0.000114
6     0.000123
7     0.000000
8     0.000000
9     0.000000
10    0.000000
11    0.000000
12    0.000000
13    0.000000
dtype: float64